In [2]:
#Download the required libraries
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.onnx
import torchvision

%matplotlib inline

plt.ion()   # interactive mode

**ЗАГРУЗКА ДАННЫХ**

In [6]:
#Получаю адрес папки /content (папка диска)
dir = os.path.abspath(os.curdir)
# указываем путь до датасета 
data_dir=os.path.join(dir, "DATASET")

# Масштабируем данные до 224*224 и нормализуем для train и test
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

#Создаю функцию для загрузки изображений из датасета и ресайза для модели Create function to get your(my) images dataset and resize it to size for model
def get_dataset(data_dir, data_transforms):

    # Создаю train и test датасеты
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                              data_transforms[x])
                      for x in ['train', 'test']}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                                 shuffle=True, num_workers=4)
                  for x in ['train', 'test']}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
    #Получаем классы из имен папок train датасета
    classes = image_datasets['train'].classes

    return dataloaders["train"], dataloaders['test'], classes, dataset_sizes

In [7]:
# Загружаем данные train и test
trainloader, testloader, classes, dataset_sizes=get_dataset(data_dir,data_transforms)
print('Классов: ',  classes)
print('Набор данных содержит: ',  dataset_sizes ," снимков")

Классов:  ['0', '2', '3', '4', '5']
Набор данных содержит:  {'train': 189, 'test': 41}  снимков


**ПРЕДПОДГОТОВКА ИЗОБРАЖЕНИЙ**

In [8]:
# Функция для создания нового изображения после трансформации 
def imshow_DenseNet(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # пауза для обновления графика

**СОЗДАНИЕ МОДЕЛИ**

In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Каждая эпоха имеет свою фазу обучения и проверки
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Перебираем данные
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Обнуляем градиент
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + оптимизатор только в тренировочной фазе
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # статистика
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [1]:
dataloaders = {'train': trainloader, 'test': testloader}
# model_ft = models.resnet18(pretrained=True) - было
model_ft = torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=True)
num_ftrs = model_ft.classifier.in_features
# Здесь размер каждого выходного образца устанавливается равным числу классов.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.classifier = nn.Sequential(nn.Linear(num_ftrs, 256), 
                                    nn.ReLU(), 
                                    nn.Dropout(0.4), 
                                    nn.Linear(256, len(classes))) 

device = 'cuda'
model_ft = model_ft.to(device)

model_ft = torch.nn.DataParallel(model_ft) # Магия параллелизма на нескоьких GPU

criterion = nn.CrossEntropyLoss()

# Оптимизируем все параметры
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Снижение LR каждые 7 эпох 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

NameError: name 'trainloader' is not defined

**ОБУЧЕНИЕ МОДЕЛИ**

In [14]:
#Train the model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=50)

Epoch 0/49
----------
train Loss: 1.5653 Acc: 0.2857
test Loss: 1.8114 Acc: 0.1463

Epoch 1/49
----------
train Loss: 1.4965 Acc: 0.3228
test Loss: 1.6440 Acc: 0.1951

Epoch 2/49
----------
train Loss: 1.5065 Acc: 0.3545
test Loss: 1.6412 Acc: 0.1707

Epoch 3/49
----------
train Loss: 1.5068 Acc: 0.3280
test Loss: 1.6990 Acc: 0.1951

Epoch 4/49
----------
train Loss: 1.4883 Acc: 0.3175
test Loss: 1.7750 Acc: 0.1951

Epoch 5/49
----------
train Loss: 1.4912 Acc: 0.3492
test Loss: 1.7468 Acc: 0.1951

Epoch 6/49
----------
train Loss: 1.5015 Acc: 0.3122
test Loss: 1.6308 Acc: 0.1951

Epoch 7/49
----------
train Loss: 1.5040 Acc: 0.3333
test Loss: 1.6331 Acc: 0.1951

Epoch 8/49
----------
train Loss: 1.4842 Acc: 0.3439
test Loss: 1.6431 Acc: 0.1951

Epoch 9/49
----------
train Loss: 1.4530 Acc: 0.3651
test Loss: 1.6813 Acc: 0.1951

Epoch 10/49
----------
train Loss: 1.4830 Acc: 0.3228
test Loss: 1.7013 Acc: 0.1951

Epoch 11/49
----------
train Loss: 1.4712 Acc: 0.3439
test Loss: 1.7214 Acc

**РЕЗУЛЬТАТЫ МОДЕЛИ**

In [15]:
# Визуализируем результаты
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = model_ft(images)
        _, predicted = torch.max(outputs.data, 1)
        
        for printdata in list(zip(predicted,labels,outputs)):
            printclass =[classes[int(printdata[0])],classes[int(printdata[1])]]
            print('Predict class - {0}, real class - {1}, probability ({2},{3}) - {4}'.format( printclass[0],printclass[1],
                                                                              classes[0], classes [1],printdata[2]))

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # imshow_DenseNet(torchvision.utils.make_grid(images))
        #print('GroundTruth: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))
print('Accuracy of the network on the', dataset_sizes['test'], 'test images: %d %%' % (
    100 * correct / total))

Predict class - 2, real class - 0, probability (0,2) - tensor([-0.6284,  0.4675, -0.1282,  0.4590,  0.1947], device='cuda:0')
Predict class - 4, real class - 2, probability (0,2) - tensor([-0.6454,  0.3062, -0.0745,  0.4121,  0.0049], device='cuda:0')
Predict class - 2, real class - 3, probability (0,2) - tensor([-0.7814,  0.4678, -0.0424,  0.4316,  0.0075], device='cuda:0')
Predict class - 4, real class - 4, probability (0,2) - tensor([-0.8019,  0.4400, -0.0931,  0.5725, -0.0546], device='cuda:0')
Predict class - 4, real class - 4, probability (0,2) - tensor([-0.8369,  0.4122, -0.1063,  0.5873, -0.0555], device='cuda:0')
Predict class - 2, real class - 2, probability (0,2) - tensor([-0.6609,  0.4966, -0.0399,  0.4633, -0.0056], device='cuda:0')
Predict class - 4, real class - 2, probability (0,2) - tensor([-0.8048,  0.4505, -0.1365,  0.4953, -0.0204], device='cuda:0')
Predict class - 4, real class - 2, probability (0,2) - tensor([-0.8835,  0.4635, -0.1303,  0.5236,  0.0615], device='c

**СОХРАНЯЕМ МОДЕЛЬ**

In [16]:
# Сохраним обученную модель
dir=''
PATH =os.path.join(dir, "model v0.1(CT)K")
torch.save(model_ft, PATH)

**ИСПОЛЬЗУЕМ ОБУЧЕННУЮ МОДЕЛЬ ДЛЯ ПРЕДСКАЗАНИЙ НА РАБОЧИХ ДАННЫХ**

In [17]:
modelBR=torch.load('model v0.1(CT)K')

In [18]:
dir=''
PATH = os.path.join(dir, "modelKK.pt")
torch.save(modelBR.state_dict(), PATH)

Загружаем и преобразуем изображение в модель

In [22]:
path_file = 'DATASET'
transforms_file = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
image_datasets = datasets.ImageFolder(os.path.join(path_file,'test'), transforms_file)
workloader2 = torch.utils.data.DataLoader(image_datasets, batch_size=1, shuffle=True, num_workers=4)                                          

In [19]:
!ls

bi-rads-classifier		     DATASET-20200616T230435Z-005.zip
bi-rads-classifier.rar		     modelKK.pt
BI_RADS_Classifier_v0_2_2(CT).ipynb  model v0.1(CT)K
DATASET


In [23]:
# Предсказываем класс
classes_test = ['0', '1', '2', '3', '4', '5']
with torch.no_grad():
    for data in workloader2:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = modelBR(images)
        _, predicted = torch.max(outputs.data, 1)
        for printdata in list(zip(predicted,labels,outputs)):
            printclass =[classes_test[int(printdata[0])],classes_test[int(printdata[1])]]
            print(u'Предсказанный класс по классификации BI-RADS - {0}'.format(printclass[0]))

Предсказанный класс по классификации BI-RADS - 1
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс по классификации BI-RADS - 1
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс по классификации BI-RADS - 1
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс по классификации BI-RADS - 1
Предсказанный класс по классификации BI-RADS - 1
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс по классификации BI-RADS - 1
Предсказанный класс по классификации BI-RADS - 3
Предсказанный класс 